# TopicGPT_Python package

`topicgpt_python` consists of five modules in total: 
- `generate_topic_lvl1` generates high-level and generalizable topics. 
- `generate_topic_lvl2` generates low-level and specific topics to each high-level topic.
- `refine_topics` refines the generated topics by merging similar topics and removing irrelevant topics.
- `assign_topics` assigns the generated topics to the input text, along with a quote that supports the assignment.
- `correct_topics` corrects the generated topics by reprompting the model so that the topic assignment is grounded in the topic list. 

![topicgpt_python](assets/img/pipeline.png)

## Setup
1. Make a new Python 3.9+ environment using virtualenv or conda. 
2. Install the required packages: `pip install --upgrade topicgpt_python`.
- Our package supports OpenAI API, Google Cloud Vertex AI API, Gemini API, Azure API, and vLLM inference. vLLM requires GPUs to run. 
- Please refer to https://openai.com/pricing/ for OpenAI API pricing or to https://cloud.google.com/vertex-ai/pricing for Vertex API pricing. 

In [ ]:
# Run in shell
!pip install --upgrade topicgpt_python

# Needed only for the OpenAI API deployment
export OPENAI_API_KEY={your_openai_api_key}

# Needed only for the Vertex AI deployment
export VERTEX_PROJECT={your_vertex_project}   # e.g. my-project
export VERTEX_LOCATION={your_vertex_location} # e.g. us-central1

# Needed only for Gemini deployment
export GEMINI_API_KEY={your_gemini_api_key}

# Needed only for the Azure API deployment
export AZURE_OPENAI_API_KEY={your_azure_api_key}
export AZURE_OPENAI_ENDPOINT={your_azure_endpoint}

## Usage
1. First, define the necessary file paths for I/O operations in `config.yml`. 
2. Then, import the necessary modules and functions from `topicgpt_python`.
3. Store your data in `data/input` and modify the `data_sample` path in `config.yml`. 

- Prepare your `.jsonl` data file in the following format:
    ```
    {
        "id": "IDs (optional)",
        "text": "Documents",
        "label": "Ground-truth labels (optional)"
    }
    ```

In [1]:
from topicgpt_python import *
import yaml

with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

WARNING 11-10 10:22:32 _custom_ops.py:19] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 11-10 10:22:32 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


### Topic Generation 
Generate high-level topics using `generate_topic_lvl1`. 
- Define the api type and model. 
- Define your seed topics in `prompt/seed_1.md`.
- (Optional) Modify few-shot examples in `prompt/generation_1.txt`.
- Expect the generated topics in `data/output/{data_name}/generation_1.md` and `data/output/{data_name}/generation_1.jsonl`.
- Right now, early stopping is set to 100, meaning that if no new topic has been generated in the last 100 iterations, the generation process will stop.

In [2]:
generate_topic_lvl1(
    "openai",
    "gpt-4o",
    config["data_sample"],
    config["generation"]["prompt"],
    config["generation"]["seed"],
    config["generation"]["output"],
    config["generation"]["topic_output"],
    verbose=config["verbose"],
)

-------------------
Initializing topic generation...
Model: gpt-4o
Data file: data/input/sample.jsonl
Prompt file: prompt/generation_1.txt
Seed file: prompt/seed_1.md
Output file: data/output/sample/generation_1.jsonl
Topic file: data/output/sample/generation_1.md
-------------------


 20%|██        | 1/5 [00:02<00:11,  2.86s/it]

Prompt token usage: 610 ~$0.0030499999999999998
Response token usage: 18 ~$0.00027
Topics: [1] Environment: Involves the management and conservation of natural resources and ecosystems.
--------------------


 40%|████      | 2/5 [00:04<00:06,  2.05s/it]

Prompt token usage: 1271 ~$0.0063549999999999995
Response token usage: 24 ~$0.00036
Topics: [1] Environment: Mentions the use of land and natural resources, including hydropower generation and land management.
--------------------


...

Prompt token usage: 823 ~$0.004115
Response token usage: 17 ~$0.000255
Topics: [1] Environment: Mentions the protection and management of natural habitats and ecosystems.
--------------------


 80%|████████  | 4/5 [00:06<00:01,  1.45s/it]

Prompt token usage: 632 ~$0.0031599999999999996
Response token usage: 14 ~$0.00021
Topics: [1] Environment: Mentions sustainable development and environmental review processes.
--------------------


100%|██████████| 5/5 [00:07<00:00,  1.57s/it]

Prompt token usage: 566 ~$0.00283
Response token usage: 17 ~$0.000255
Topics: [1] Trade: Mentions the exchange of capital, goods, and services.
--------------------


### Topic Refinement
If topics are generated by a weaker model, there sometimes exist irrelevant or redundant topics. This module: 
- Merges similar topics.
- Removes overly specific or redundant topics that occur < 1% of the time (you can skip this by setting `remove` to False in `config.yml`).
- Expect the refined topics in `data/output/{data_name}/refinement_1.md` and `data/output/{data_name}/refinement_1.jsonl`. If nothing happens, it means that the topic list is coherent.
- If you're unsatisfied with the refined topics, call the function again with the refined topic file and refined topic file from the previous iteration

In [ ]:
# Optional: Refine topics if needed
if config["refining_topics"]:
    refine_topics(
        "openai",
        "gpt-4o",
        config["refinement"]["prompt"],
        config["generation"]["output"],
        config["generation"]["topic_output"],
        config["refinement"]["topic_output"],
        config["refinement"]["output"],
        verbose=config["verbose"],
        remove=config["refinement"]["remove"],
        mapping_file=config["refinement"]["mapping_file"]
    )

-------------------
Initializing topic refinement...
Model: gpt-4o
Input data file: data/output/sample/generation_1.jsonl
Prompt file: prompt/refinement.txt
Output file: data/output/sample/refinement.md
Topic file: data/output/sample/generation_1.md
-------------------
No topic pairs to be merged.
No topics removed.
Node('/Topics', count=1, desc='Root topic', lvl=0)
├── Node('/Topics/Environment', count=4, desc='Involves the management and conservation of natural resources and ecosystems.', lvl=1)
└── Node('/Topics/Trade', count=1, desc='Mentions the exchange of capital, goods, and services.', lvl=1)


### Subtopic Generation 
Generate subtopics using `generate_topic_lvl2`.
- This function iterates over each high-level topic and generates subtopics based on a few example documents associated with the high-level topic.
- Expect the generated topics in `data/output/{data_name}/generation_2.md` and `data/output/{data_name}/generation_2.jsonl`.

In [4]:
# Optional: Generate subtopics
if config["generate_subtopics"]:
    generate_topic_lvl2(
        "openai",
        "gpt-4o",
        config["generation"]["topic_output"],
        config["generation"]["output"],
        config["generation_2"]["prompt"],
        config["generation_2"]["output"],
        config["generation_2"]["topic_output"],
        verbose=config["verbose"],
    )

-------------------
Initializing topic generation (lvl 2)...
Model: gpt-4o
Data file: data/output/sample/generation_1.jsonl
Prompt file: prompt/generation_2.txt
Seed file: data/output/sample/generation_1.md
Output file: data/output/sample/generation_2.jsonl
Topic file: data/output/sample/generation_2.md
-------------------
Number of remaining documents for prompting: 5


  0%|          | 0/2 [00:00<?, ?it/s]

Current topic: [1] Environment


 50%|█████     | 1/2 [00:03<00:03,  3.28s/it]

Subtopics: [1] Environment
   [2] Conservation (Document: 1): Focuses on the preservation and management of natural areas, such as roadless areas within national forests.
   [2] Indigenous Rights and Compensation (Document: 2): Pertains to the rights and compensation of indigenous tribes for the use of their land, particularly in relation to hydropower projects.
   [2] Marine Habitat Protection (Document: 3): Involves the protection and management of marine habitats, specifically related to the conversion of decommissioned oil and gas platforms into artificial reefs.
   [2] Sustainable Transportation Development (Document: 4): Concerns the development of sustainable transportation systems, including aerotropolis projects that integrate multimodal transportation networks.
Conservation (Count: 0): Focuses on the preservation and management of natural areas, such as roadless areas within national forests.
Indigenous Rights and Compensation (Count: 0): Pertains to the rights and compensati

100%|██████████| 2/2 [00:04<00:00,  2.05s/it]

Subtopics: [1] Trade

The provided document does not relate to the topic of "Trade" or suggest any subtopics under it. Therefore, no second-level topics are added.
Not a match: 
Not a match: The provided document does not relate to the topic of "Trade" or suggest any subtopics under it. Therefore, no second-level topics are added.
--------------------------------------------------


### Topic Assignment
Assign the generated topics to the input text using `assign_topics`. Each assignment is supported by a quote from the input text.
- Expect the assigned topics in `data/output/{data_name}/assignment.jsonl`. 
- The model used here is often a weaker model to save cost, so the topics may not be grounded in the topic list. To correct this, use the `correct_topics` module. If there are still errors/hallucinations, run the `correct_topics` module again.

In [5]:
# Assignment
assign_topics(
    "openai",
    "gpt-4o-mini",
    config["data_sample"],
    config["assignment"]["prompt"],
    config["assignment"]["output"],
    config["generation"][
        "topic_output"
    ],  # TODO: change to generation_2 if you have subtopics, or config['refinement']['topic_output'] if you refined topics
    verbose=config["verbose"],
)

-------------------
Initializing topic assignment...
Model: gpt-4o-mini
Data file: data/input/sample.jsonl
Prompt file: prompt/assignment.txt
Output file: data/output/sample/assignment.jsonl
Topic file: data/output/sample/generation_1.md
-------------------


 20%|██        | 1/5 [00:01<00:04,  1.20s/it]

Prompt token usage: 502 ~$0.0025099999999999996
Response token usage: 49 ~$0.000735
Response: [1] Environment: The document discusses the management and conservation of roadless areas within the National Forest System, which directly relates to environmental conservation efforts. ("...directs the Secretary of Agriculture to manage such Areas to maintain their roadless character.")
--------------------


 40%|████      | 2/5 [00:02<00:04,  1.46s/it]

Prompt token usage: 1158 ~$0.00579
Response token usage: 113 ~$0.001695
Response: [1] Environment: The document discusses the use of land for hydropower generation and the management of resources related to the Spokane Tribe, which involves environmental considerations. ("...for the use of its land for hydropower generation by the Grand Coulee Dam.")

[1] Trade: The document mentions financial transactions and compensation related to the use of resources, which can be associated with trade aspects. ("...constitute full satisfaction of the claim of the Tribe to a fair share of the annual hydropower revenues generated by the Grand Coulee Dam project...")
--------------------


...

Prompt token usage: 710 ~$0.00355
Response token usage: 68 ~$0.00102
Response: [1] Environment: The document discusses the assessment and management of offshore oil and gas platforms that have become critical for marine fisheries habitat, which involves the conservation of ecosystems. ("...assess each offshore oil and gas platform in the Gulf of Mexico that is no longer useful for operations, and has become critical for a marine fisheries habitat...")
--------------------


 80%|████████  | 4/5 [00:06<00:01,  1.63s/it]

Prompt token usage: 519 ~$0.002595
Response token usage: 113 ~$0.001695
Response: [1] Environment: The document discusses the establishment of a grant program that includes activities related to environmental review, which is part of managing and conserving natural resources. ("...assist in planning, design, environmental review, or land acquisition activities for one or more specified kinds of projects...")

[1] Trade: The document mentions the development of transportation systems that facilitate efficient and sustainable connectivity, which can be related to the exchange of goods and services. ("...provide efficient, sustainable, and intermodal connectivity to a defined region of economic significance centered around a major airport.")
--------------------


100%|██████████| 5/5 [00:07<00:00,  1.51s/it]

Prompt token usage: 453 ~$0.002265
Response token usage: 71 ~$0.001065
Response: [1] Trade: The document discusses the requirements for issuing driver's licenses and identification documents, which relates to the verification of lawful immigration status, impacting the exchange of services and identification in trade contexts. ("...prohibit a state from issuing a driver's license or identification document to a person unless the state has complied with certain citizenship or lawful immigration status verification requirements.")
--------------------


In [6]:
# Correction
correct_topics(
    "openai",
    "gpt-4o-mini",
    config["assignment"]["output"],
    config["correction"]["prompt"],
    config["generation"][
        "topic_output"
    ],  # TODO: change to generation_2 if you have subtopics, or config['refinement']['topic_output'] if you refined topics
    config["correction"]["output"],
    verbose=config["verbose"],
)

-------------------
Initializing topic correction...
Model: gpt-4o-mini
Data file: data/output/sample/assignment.jsonl
Prompt file: prompt/correction.txt
Output file: data/output/sample/assignment_corrected.jsonl
Topic file: data/output/sample/generation_1.md
-------------------
Number of errors: 0
Number of hallucinated topics: 0
All topics are correct.
